In [1]:
# !unzip /path/to/ngrok.zip

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ngrok config add-authtoken 비밀입니다

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
%%capture
!pip install fastapi
!pip install uvicorn
!pip install pyngrok
!pip install python-multipart
from fastapi import FastAPI, UploadFile, File
from pyngrok import ngrok
import uvicorn
import threading
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [5]:
app = FastAPI(docs_url="/docs")

In [6]:
# 모델 불러오기
model_path = '/content/drive/MyDrive/SW 전문인재양성 2기/개인프로젝트/CNN/bottle/cnn.h5'
model = load_model(model_path)

In [7]:
# 이미지 전처리 함수
def preprocess_image(image):
    # 이미지 크기 조정
    image = image.resize((224, 224))
    # 이미지를 배열로 변환
    image_array = img_to_array(image)
    # 이미지를 모델에 입력하기 위해 차원 확장
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

In [8]:
# 예측 함수
def predict(image):
    preprocessed_image = preprocess_image(image)
    # 모델을 사용하여 예측
    predictions = model.predict(preprocessed_image)
    # 결과를 'good' 또는 'bad'로 변환
    label = 'good' if predictions[0][0] > 0.5 else 'bad'
    return label

In [9]:
# FastAPI 엔드포인트
@app.get("/")
def read_root():
    return {"Hello": "World"}


In [10]:
# FastAPI 엔드포인트 - 이미지 예측
@app.post("/predict/")
async def predict_image(file: UploadFile = File(...)):
    try:
        # 업로드된 파일을 PIL 이미지로 변환
        image = Image.open(file.file)
        # 예측
        label = predict(image)
        # 결과 반환
        return {"prediction": label}
    except Exception as e:
        return {"error": str(e)}


In [11]:
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [13]:
# ngrok을 사용하여 포트 8000을 터널링하고 인터넷에 노출
url = ngrok.connect(8000)
print('Public URL:', url)

Public URL: NgrokTunnel: "https://7df6-34-74-79-243.ngrok-free.app" -> "http://localhost:8000"


In [14]:
# FastAPI 실행
# FastAPI 실행
threading.Thread(target=run).start()

INFO:     Started server process [7593]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [ ]:
!lsof -i :8000

In [ ]:
# !kill -9